In [1]:
import pandas as pd
import numpy as np
import sys 
sys.path.insert(1, 'c:/xxx')
from eda import eda
df=pd.read_csv('c:/xxx/assessment_input.csv')
df = df.drop(df.columns[-1],axis=1)
df['Time'] = pd.to_datetime(df['Time'],format='%d/%m/%Y %H:%M') #28/04/2009 0:00
df.replace('#VALUE!',np.nan, inplace=True)
skiplist=['Time','PLANT']


###FOR PLANT D05####
df=df[df['PLANT']=='P05']

for i in df.columns:
   if i not in skiplist:
      df[i]=pd.to_numeric(df[i], errors='coerce')
      df[i].fillna(value=df[i].mean(), inplace=True)
      df[i].replace(0, value=df[i].mean(), inplace=True)
    #data=data.replace(0,mean_artist_hotness) 

# df['D']=pd.to_numeric(df['D'], errors='coerce')
# df['D'].fillna(value=df['D'].mean(), inplace=True)    
df

ModuleNotFoundError: No module named 'missingno'

In [ ]:
#df.to_csv('c:/xxx/df_p05.csv')
eda(df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

figsize=(10,5)
for s in df.columns:
    if s not in skiplist:
        if s=='DOSE':
            ax=df.plot(x='Time',y=s,figsize=(20,10), label=s)
        else:
            df.plot(x='Time',y=s,ax=ax)

plt.show()


In [ ]:
data =df.drop(['Time'],axis=1)
data.index=df['Time']

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
data1=data.drop(['PLANT'],axis=1)
np.around(coint_johansen(data1,-1,1).eig,3)

In [ ]:
#creating the train and validation set
train = data1[:int(0.8*(len(data1)))].astype(float)
valid = data1[int(0.8*(len(data1))):].astype(float)

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
#converting predictions to dataframe
cols = data1.columns
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,11):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', sqrt(mean_squared_error(pred[i], valid[i])))

In [ ]:
#make final predictions
forecast_time = 200
model = VAR(endog=data1)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=forecast_time)
print(yhat)

In [ ]:
#https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=200)
yhat

In [ ]:
pred_df=pd.DataFrame(yhat)
pred_df.columns=cols
idx=data1.tail().index[0]
newlist=[]

for i in range(0,forecast_time):
     idx = idx + pd.Timedelta('12hour')
     newlist.append(idx)

pred_df['Time']=newlist
pred_df.index=pred_df['Time']
pred_df.drop(['Time'], axis=1)
pred_df


In [ ]:
#plot data1 + pred_df
#highlight from pred_df.start to end
fig = plt.figure(figsize=(20,5))
ax  = fig.add_subplot(111)
fig.suptitle('Actual vs Predict for DOSE')
# plt.plot(data1['DOSE'], label='actual')
# plt.plot(pred_df['DOSE'], label='predict')
ax =  data1.plot(y='DOSE',ax=ax, label='Actual')
ax = pred_df.plot(y='DOSE',ax=ax, label='Predict')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

figsize=(20,10)
for s in data1.columns:
    if s not in skiplist:
        if s=='DOSE':
            ax = data1.plot(y=s, label=s + ' Actual')
            ax = pred_df.plot(y=s,ax=ax, label=s + ' Predict')
        else:
            ax = data1.plot(y=s, label=s + ' Actual')
            ax = pred_df.plot(y=s,ax=ax, label=s + ' Predict')

plt.show()